In [1]:
import napari
import napari_stress
from napari_stress import measurements, approximation
import matplotlib.pyplot as plt

import numpy as np

In [2]:
# Get some sample data
pointcloud = napari_stress.get_droplet_point_cloud()[0]

In [3]:
viewer = napari.Viewer(ndisplay=3)

Assistant skips harvesting pyclesperanto as it's not installed.


22-Aug-22 14:22:37 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 2180x1370+1592+448 (frame: 2206x1441+1579+390) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY4". Resulting geometry: 4366x2119+1597+475 (frame: 4392x2190+1584+417) margins: 13, 58, 13, 13 minimum size: 608x557 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=634,628 maxtrack=0,0)


In [4]:
layer_raw = viewer.add_layer(napari.layers.Layer.create(data = pointcloud[0], meta=pointcloud[1], layer_type=pointcloud[2]))

In [5]:
max_degree = 5

## Ellipsoid fit

In [6]:
fitted_ellipsoid = approximation.least_squares_ellipsoid(pointcloud[0])
fitted_ellipse_points = approximation.expand_points_on_ellipse(fitted_ellipsoid, pointcloud[0])

In [7]:
_,ellipsoid_features, ellipsoid_metadata = measurements.curvature_on_ellipsoid(fitted_ellipsoid, fitted_ellipse_points)

In [8]:
mean_curvature_ellipsoid = ellipsoid_features['mean_curvature']
H_ellipsoid_major_medial_minor = ellipsoid_metadata['H_ellipsoid_major_medial_minor']

In [9]:
viewer.add_vectors(fitted_ellipsoid)
viewer.add_points(fitted_ellipse_points, features = ellipsoid_features, face_color = 'mean_curvature', size=0.5, face_colormap = 'jet')

<Points layer 'fitted_ellipse_points' at 0x2041746eb20>

Now do a spherical harmonics expansion of the ellipsoid points:

In [10]:
fitted_ellipse_points_spherical_harmoniocs = napari_stress.fit_spherical_harmonics(fitted_ellipse_points, max_degree=max_degree)
coefficients = fitted_ellipse_points_spherical_harmoniocs[1]['metadata']['spherical_harmonics_coefficients']
quadrature_points, lebedev_fit = napari_stress.lebedev_quadrature(coefficients=coefficients, number_of_quadrature_points=500, use_minimal_point_set=False)
manifold = napari_stress.create_manifold(quadrature_points, lebedev_fit=lebedev_fit, max_degree=max_degree)

In [11]:
_, features, metadata = measurements.calculate_mean_curvature_on_manifold(manifold)
print(features.keys())
print(metadata.keys())

Old: <class 'napari_stress._stress.manifold_SPB.manifold'> New: napari.layers.Points
dict_keys(['Mean_curvature_at_lebedev_points'])
dict_keys(['H0_arithmetic_average', 'H0_surface_integral'])


In [12]:
H0_surface_integral_ellipsoid = metadata['H0_surface_integral']
mean_curvature_ellipsoid = features['Mean_curvature_at_lebedev_points']

## Spherical harmonics expansion

In [13]:
fitted_points = napari_stress.fit_spherical_harmonics(pointcloud[0], max_degree=max_degree)
fitted_points_layer = viewer.add_points(fitted_points[0], **fitted_points[1], name='spherical harmonics expansion')

In [14]:
coefficients = fitted_points[1]['metadata']['spherical_harmonics_coefficients']
quadrature_points, lebedev_fit = napari_stress.lebedev_quadrature(coefficients=coefficients, number_of_quadrature_points=500, use_minimal_point_set=False)
manifold = napari_stress.create_manifold(quadrature_points, lebedev_fit=lebedev_fit, max_degree=max_degree)

In [15]:
_, features, metadata = measurements.calculate_mean_curvature_on_manifold(manifold)
print(features.keys())
print(metadata.keys())

Old: <class 'napari_stress._stress.manifold_SPB.manifold'> New: napari.layers.Points
dict_keys(['Mean_curvature_at_lebedev_points'])
dict_keys(['H0_arithmetic_average', 'H0_surface_integral'])


In [16]:
H0_surface_integral_spherical_harmonics = metadata['H0_surface_integral']
mean_curvature_spherical_harmonics = features['Mean_curvature_at_lebedev_points']

## Stresses

In [17]:
gamma = 0.5  # N/m

### Tissue-scale

In [18]:
# use H0_Ellpsoid to calculate tissue stress projections:
sigma_11_e = 2 * gamma * (H_ellipsoid_major_medial_minor[0] - H0_surface_integral_ellipsoid)
sigma_22_e = 2 * gamma * (H_ellipsoid_major_medial_minor[1] - H0_surface_integral_ellipsoid)
sigma_33_e = 2 * gamma * (H_ellipsoid_major_medial_minor[2] - H0_surface_integral_ellipsoid)

In [19]:
# tissue stress tensor (elliptical coordinates)
Tissue_Stress_Tensor_elliptical = np.zeros((3,3))
Tissue_Stress_Tensor_elliptical[0,0] = sigma_11_e
Tissue_Stress_Tensor_elliptical[1,1] = sigma_22_e
Tissue_Stress_Tensor_elliptical[2,2] = sigma_33_e
Tissue_Stress_Tensor_elliptical

array([[ 0.01600191,  0.        ,  0.        ],
       [ 0.        , -0.00048086,  0.        ],
       [ 0.        ,  0.        , -0.01086109]])

In [20]:
# get rotation matrix:
Rotation_matrix = fitted_ellipsoid[:, 1].T  # the normalized major/minor axis vectors used as column vectors compose the orientation matrix of the ellipsoid
Rotation_matrix = Rotation_matrix / np.linalg.norm(Rotation_matrix, axis=0)

To prove that this is a rotation matrix we multiply it with its inverse - this should return the identiy matrix

In [21]:
np.dot(Rotation_matrix, Rotation_matrix.T)

array([[ 1.00000000e+00,  3.06750253e-16, -3.49196291e-17],
       [ 3.06750253e-16,  1.00000000e+00, -2.21854909e-17],
       [-3.49196291e-17, -2.21854909e-17,  1.00000000e+00]])

In [22]:
# cartesian tissue stress tensor:
basis_matrix = fitted_ellipsoid
Tissue_Stress_Tensor_cartesian = np.dot( np.dot(Rotation_matrix.T ,Tissue_Stress_Tensor_elliptical), Rotation_matrix)
sigma_11_tissue_x = Tissue_Stress_Tensor_cartesian[0,0]
sigma_22_tissue_y = Tissue_Stress_Tensor_cartesian[1,1]
sigma_33_tissue_z = Tissue_Stress_Tensor_cartesian[2,2]
Tissue_Stress_Tensor_cartesian

array([[ 0.00201583, -0.00479792, -0.00491557],
       [-0.00479792,  0.00872738,  0.00901813],
       [-0.00491557,  0.00901813, -0.00608324]])

### Cell-scale

In [23]:
# get quadrature points on ellipsoid surface
quadrature_points_on_ellipsoid = approximation.expand_points_on_ellipse(fitted_ellipsoid, quadrature_points)
_, features, metadata = measurements.curvature_on_ellipsoid(fitted_ellipsoid, quadrature_points_on_ellipsoid)
mean_curvature_ellipsoid_lebedev_points = features['mean_curvature']

In [24]:
anisotropic_stress_cell = 2 * gamma * (mean_curvature_spherical_harmonics - H0_surface_integral_spherical_harmonics)
anisotropic_stress_quadrature_points_cell = 2 * gamma * (mean_curvature_spherical_harmonics - mean_curvature_ellipsoid_lebedev_points - 
                                                        (H0_surface_integral_spherical_harmonics- H0_surface_integral_ellipsoid))
anisotropic_stress_tissue_quadrature_points = 2 * gamma * (mean_curvature_ellipsoid_lebedev_points - H0_surface_integral_ellipsoid)